#Telecom Domain ReadOps Assignment
This notebook contains assignments to practice Spark read options and Databricks volumes. <br>
Sections: Sample data creation, Catalog & Volume creation, Copying data into Volumes, Path glob/recursive reads, toDF() column renaming variants, inferSchema/header/separator experiments, and exercises.<br>

![](https://fplogoimages.withfloats.com/actual/68009c3a43430aff8a30419d.png)
![](https://theciotimes.com/wp-content/uploads/2021/03/TELECOM1.jpg)

##First Import all required libraries & Create spark session object

##1. Write SQL statements to create:
1. A catalog named telecom_catalog_assign
2. A schema landing_zone
3. A volume landing_vol
4. Using dbutils.fs.mkdirs, create folders:<br>
/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/
/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/
/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/
5. Explain the difference between (Just google and understand why we are going for volume concept for prod ready systems):<br>
a. Volume vs DBFS/FileStore<br>
b. Why production teams prefer Volumes for regulated data<br>

In [0]:
from pyspark.sql.session import SparkSession
print(spark)
spark1=SparkSession.builder.getOrCreate()
print(spark1)

In [0]:
%sql
--create catalog if not exists telecom_catalog_assign;
create schema if not exists telecom_catalog_assign.landing_zone;
create volume if not exists telecom_catalog_assign.landing_zone.landing_vol;

In [0]:
base_path = "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/"
cust_path = dbutils.fs.mkdirs(base_path + "customer/")
usage_Path = dbutils.fs.mkdirs(base_path + "usage/")
tower_path = dbutils.fs.mkdirs(base_path + "tower/")


- DBFS/FileStore → convenient, workspace-level file storage for quick sharing or temporary datasets.
- Volumes → governed, catalog-aware storage for production data pipelines with security and compliance.

#### Volumes are the better fit for production pipelines, while DBFS/FileStore is more for scratchpad work or demos


In [0]:
#this data was given 
##Data files to use in this usecase:
customer_csv = '''
101,Arun,31,Chennai,PREPAID
102,Meera,45,Bangalore,POSTPAID
103,Irfan,29,Hyderabad,PREPAID
104,Raj,52,Mumbai,POSTPAID
105,,27,Delhi,PREPAID
106,Sneha,abc,Pune,PREPAID
'''

usage_tsv = '''customer_id\tvoice_mins\tdata_mb\tsms_count
101\t320\t1500\t20
102\t120\t4000\t5
103\t540\t600\t52
104\t45\t200\t2
105\t0\t0\t0
'''

tower_logs_region1 = '''event_id|customer_id|tower_id|signal_strength|timestamp
5001|101|TWR01|-80|2025-01-10 10:21:54
5004|104|TWR05|-75|2025-01-10 11:01:12
'''
tower_logs_region2 = '''event_id|customer_id|tower_id|signal_strength|timestamp
5002|102|TWR02|-85|2025-01-10 10:21:54
5003|103|TWR03|-80|2025-01-10 10:21:54
5005|105|TWR04|-70|2025-01-10 11:01:12'''

##2. Filesystem operations
1. Write code to copy the above datasets into your created Volume folders:
Customer → /Volumes/.../customer/
Usage → /Volumes/.../usage/
Tower (region-based) → /Volumes/.../tower/region1/ and /Volumes/.../tower/region2/

2. Write a command to validate whether files were successfully copied

In [0]:
dbutils.fs.put("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer_csv",customer_csv,overwrite=True)
dbutils.fs.put("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage_tsv",usage_tsv,overwrite=True)
dbutils.fs.put("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_logs_r1",tower_logs_region1,overwrite=True)
dbutils.fs.put("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_logs_r2",tower_logs_region2,overwrite=True)

dbutils.fs.ls("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/")

##3. Directory Read Use Cases
1. Read all tower logs using:
Path glob filter (example: *.csv)
Multiple paths input
Recursive lookup

2. Demonstrate these 3 reads separately:
Using pathGlobFilter
Using list of paths in spark.read.csv([path1, path2])
Using .option("recursiveFileLookup","true")

3. Compare the outputs and understand when each should be used.

In [0]:
tower_df1 = spark.read.csv(path=["/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/"],header=True,inferSchema=True,sep="|",pathGlobFilter="tower_logs_*",recursiveFileLookup=True)
print(tower_df1.count())
display(tower_df1)
usage_df1 = spark.read.csv(path=["/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/"],header=True,inferSchema=True,sep="\t",
pathGlobFilter="usage*",recursiveFileLookup=True)
print(usage_df1.count())
display(usage_df1)
cust_df1 = spark.read.csv(path=["/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/"],pathGlobFilter="*sv",recursiveFileLookup=True,inferSchema=True,header=True,sep=",").toDF("customer_id","name","age","city","plan_type")
print(cust_df1.count())
display(cust_df1)

##4. Schema Inference, Header, and Separator
1. Try the Customer, Usage files with the option and options using read.csv and format function:<br>
header=false, inferSchema=false<br>
or<br>
header=true, inferSchema=true<br>
2. Write a note on What changed when we use header or inferSchema  with true/false?<br>
3. How schema inference handled “abc” in age?<br>

In [0]:
#1&2 are covered with true/false in header/inferSchema
cust_df2=spark.read.option("header","false").option("inferSchema","false").option("sep",",").format("csv").load("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer_csv")
#.toDF("customer_id","name","age","city","plan_type")
cust_df2.printSchema() 
cust_df2.show(2)
usage_df2=spark.read.option("header","true").option("inferSchema","true").option("sep","\t").format("csv").load("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage_tsv")
usage_df2.printSchema()
usage_df2.show(2)

#2 when inferschema = false: all the column schema is inferred as string by default
        #inferschema = True: column schema is inferred based on data & data type is assigned accordingly
        #header = false: if data has header then it is considered as 1st row of data rather as header
        #header = true: if data has header then it is considered as header row
 
#3 the age column has 'abc' which is string value, hence entire column is assumed as string when inferschema is set to true, if inferschema is set as false, by default schema is set as string for all columns.

##5. Column Renaming Usecases
1. Apply column names using string using toDF function for customer data
2. Apply column names and datatype using the schema function for usage data
3. Apply column names and datatype using the StructType with IntegerType, StringType, TimestampType and other classes for towers data 

In [0]:
from pyspark.sql.types import StructType,StructField,IntegerType,StringType,TimestampType
cust_schema = StructType([StructField("customer_id",IntegerType(),False),
                            StructField("name",StringType(),True),
                            StructField("age",IntegerType(),True),
                            StructField("city",StringType(),True),
                            StructField("Plan_type",StringType(),True)])
customer_str_struct = "cust_id integer,name string,age integer,city string, plan_type string"
tower_schema = StructType([StructField("event_id",IntegerType(),False),
                            StructField("customer_id",IntegerType(),True),
                            StructField("tower_id",StringType(),True),
                            StructField("signal_strength",IntegerType(),True),
                            StructField("timestamp",TimestampType(),True)])
#1.
cust_df3=spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer_csv",header=True,inferSchema=True).toDF("customer_id","name","age","city","plan_type")
cust_df3.printSchema() 
cust_df3.show(2)
#2.
cust_df4 = spark.read.schema(customer_str_struct).csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer_csv")
cust_df4.printSchema()
cust_df4.show(2)
#3.
tower_df2 = spark.read.schema(tower_schema).csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_logs_*")
tower_df2.printSchema()
tower_df2.show(2)
cust_df5 = spark.read.schema(cust_schema).csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer_csv")
cust_df3.printSchema()
cust_df3.show(2)


##6. Write Operations (Data Conversion/Schema migration) – CSV Format Usecases
1. Write customer data into CSV format using overwrite mode
2. Write usage data into CSV format using append mode
3. Write tower data into CSV format with header enabled and custom separator (|)
4. Read the tower data in a dataframe and show only 5 rows.
5. Download the file into local from the catalog volume location and see the data of any of the above files opening in a notepad++.

In [0]:
cust_write=cust_df1.write.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/target_cust.csv",mode="overwrite")
usage_df1.write.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/target_usage.csv",mode="append")
tower_df1.write.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/target_tower.csv",mode="overwrite",header=True,sep="|")

#read data from target and show
tower_target_show= spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/target_tower.csv",sep="|",header=True)
tower_target_show.show(5)

##7. Write Operations (Data Conversion/Schema migration)– JSON Format Usecases
1. Write customer data into JSON format using overwrite mode
2. Write usage data into JSON format using append mode and snappy compression format
3. Write tower data into JSON format using ignore mode and observe the behavior of this mode
4. Read the tower data in a dataframe and show only 5 rows.
5. Download the file into local harddisk from the catalog volume location and see the data of any of the above files opening in a notepad++.

In [0]:
cust_df1.write.json("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/target_cust.json",mode="overwrite")
usage_df1.write.json("/Volumes/telecom_catalog_assign/landing_zone/#landing_vol/usage/target_usage.json",mode="append",compression="snappy")
tower_df1.write.json("/Volumes/telecom_catalog_assign/landing_zone/#landing_vol/tower/target_tower.json",mode="ignore")

tower_target_show = spark.read.json("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/target_tower.json")
tower_target_show.show(5)


##8. Write Operations (Data Conversion/Schema migration) – Parquet Format Usecases
1. Write customer data into Parquet format using overwrite mode and in a gzip format
2. Write usage data into Parquet format using error mode
3. Write tower data into Parquet format with gzip compression option
4. Read the usage data in a dataframe and show only 5 rows.
5. Download the file into local harddisk from the catalog volume location and see the data of any of the above files opening in a notepad++.

In [0]:
cust_df1.write.parquet("/Volumes/telecom_catalog_assign/landing_zone/#landing_vol/customer/target_cust.parquet",mode="overwrite",compression="gzip")
usage_df1.write.parquet("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/target_usage.parquet",mode="error")
tower_df1.write.parquet("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/target_tower.parquet",compression="gzip")

usage_target_show= spark.read.parquet("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/target_usage.parquet")
usage_target_show.show(5)

##9. Write Operations (Data Conversion/Schema migration) – Orc Format Usecases
1. Write customer data into ORC format using overwrite mode
2. Write usage data into ORC format using append mode
3. Write tower data into ORC format and see the output file structure
4. Read the usage data in a dataframe and show only 5 rows.
5. Download the file into local harddisk from the catalog volume location and see the data of any of the above files opening in a notepad++.

In [0]:
cust_df1.write.orc("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/target_cust.orc",mode="overwrite")
usage_df1.write.orc("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/target_usage.orc",mode="append")
tower_df1.write.orc("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/target_tower.orc",mode="overwrite")

usage_target_orc=spark.read.orc("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/target_usage.orc")
usage_target_orc.show(5)


##10. Write Operations (Data Conversion/Schema migration) – Delta Format Usecases
1. Write customer data into Delta format using overwrite mode
2. Write usage data into Delta format using append mode
3. Write tower data into Delta format and see the output file structure
4. Read the usage data in a dataframe and show only 5 rows.
5. Download the file into local harddisk from the catalog volume location and see the data of any of the above files opening in a notepad++.
6. Compare the parquet location and delta location and try to understand what is the differentiating factor, as both are parquet files only.

In [0]:
cust_df1.write.format("delta").save("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/target_cust.delta",mode="overwrite")
usage_df1.write.format("delta").save("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/target_usage.delta",mode="append")
tower_df1.write.format("delta").save("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/target_tower.delta")
display(dbutils.fs.ls("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/target_tower.delta"))

usage_target_delta = spark.read.format("delta").load("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/target_usage.delta")
usage_target_delta.show(5)



##11. Write Operations (Lakehouse Usecases) – Delta table Usecases
1. Write customer data using saveAsTable() as a managed table
2. Write usage data using saveAsTable() with overwrite mode
3. Drop the managed table and verify data removal
4. Go and check the table overview and realize it is in delta format in the Catalog.
5. Use spark.read.sql to write some simple queries on the above tables created.

In [0]:
cust_df1.write.saveAsTable("telecom_catalog_assign.landing_zone.customers")
usage_df1.write.saveAsTable("telecom_catalog_assign.landing_zone.usage",mode="overwrite")
spark.sql("drop table if exists telecom_catalog_assign.landing_zone.customers")
display(spark.sql("select * from telecom_catalog_assign.landing_zone.usage"))

##12. Write Operations (Lakehouse Usecases) – Delta table Usecases
1. Write customer data using insertInto() in a new table and find the behavior
2. Write usage data using insertTable() with overwrite mode

In [0]:
spark.sql("create table if not exists telecom_catalog_assign.landing_zone.insertInto_customers using delta")
cust_df1.write.option("mergeSchema","true").insertInto("telecom_catalog_assign.landing_zone.insertInto_customers")
usage_df1.write.insertInto("telecom_catalog_assign.landing_zone.usage",overwrite=True)

##13. Write Operations (Lakehouse Usecases) – Delta table Usecases
1. Write customer data into XML format using rowTag as cust
2. Write usage data into XML format using overwrite mode with the rowTag as usage
3. Download the xml data and open the file in notepad++ and see how the xml file looks like.

In [0]:
cust_df1.write.xml("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/target_cust.xml",rowTag="cust")
usage_df1.write.xml("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/target_usage.xml",mode="overwrite",rowTag="usage")

cust_target_xml = spark.read.xml("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/target_cust.xml",rowTag="cust")
cust_target_xml.show(2)

##14. Compare all the downloaded files (csv, json, orc, parquet, delta and xml)
1. Capture the size occupied between all of these file formats and list the formats below based on the order of size from small to big.


orc and parquet are meant for huge volume of data which significantly reduce the size of the file(storage space), since below is low volume data it doesnt show the actual benefits of all file format. <br>
##### Customer data<br>
##### Sorce csv file size - 168bytes<br>
##### Target csv file size - 253bytes<br>
##### Target json file size - 523bytes<br>
##### Target orc file size - 1047bytes<br>
##### Target xml file size - 1.2kb<br>
##### Target delta file size - 1.65kb<br>
##### Target parquest file size - 2kb<br>

##15. Try to do permutation and combination of performing Schema Migration & Data Conversion operations like...
1. Read any one of the above orc data in a dataframe and write it to dbfs in a parquet format
2. Read any one of the above parquet data in a dataframe and write it to dbfs in a delta format
3. Read any one of the above delta data in a dataframe and write it to dbfs in a xml format
4. Read any one of the above delta table in a dataframe and write it to dbfs in a json format
5. Read any one of the above delta table in a dataframe and write it to another table

In [0]:
#1. Read any one of the above orc data in a dataframe and write it to dbfs in a parquet format
usage_df3 = spark.read.options(header="True",inferschema="True").format("orc").load("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/target_usage.orc")

usage_df3.write.mode("overwrite").format("parquet").save("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/target_usage_df3.parquet")

In [0]:
#2. Read any one of the above parquet data in a dataframe and write it to dbfs in a delta format
usage_df4 = spark.read.parquet("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/target_usage_df3.parquet")

usage_df4.printSchema()
usage_df4.write.mode("overwrite").format("delta").save("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/target_usage_df4.delta")


In [0]:
#3. Read any one of the above delta data in a dataframe and write it to dbfs in a xml format
usage_df5 = spark.read.format("delta").load("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/target_usage_df4.delta")

usage_df5.printSchema()
usage_df5.write.xml("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/target_usage_df5.xml",mode="overwrite",rowTag="usage")

In [0]:
#4. Read any one of the above delta table in a dataframe and write it to dbfs in a json format
tower_df3 = spark.read.format("delta").load("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/target_tower.delta/")

tower_df3.printSchema()
tower_df3.write.json("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/target_tower_df3.json",mode="overwrite")

In [0]:
#5. Read any one of the above delta table in a dataframe and write it to another table
tower_df4 = spark.read.format("delta").load("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/target_tower.delta")

tower_df4.write.xml("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/target_tower_df4.xml",mode="overwrite",rowTag="tower")

##16. Do a final exercise of defining one/two liner of...
1. When to use/benifits csv
2. When to use/benifits json<br>
###### used when data in semi-structured format with variable data format & dynamic schema or exchange of data between network or parsing of data.
3. When to use/benifit orc
4. When to use/benifit parquet
5. When to use/benifit delta<br>
###### orc/parquet/delta are serialized data formats used for large volume of data, which support optimization, space reduction and modify data(ACID properties)
6. When to use/benifit xml
7. When to use/benifit delta tables<br>
###### delta table preferred table structure of  databricks, layered on parquet file system that provides lakehouse features. 